In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt

In [3]:
# Load the CSV file
df = pd.read_csv('/kaggle/input/pad-ufes-20/PAD-UFES-20/metadata.csv')
data=df[['diagnostic','img_id']]
data.head()

,diagnostic,img_id
0,NEV,PAT_1516_1765_530.png
1,BCC,PAT_46_881_939.png
2,ACK,PAT_1545_1867_547.png
3,ACK,PAT_1989_4061_934.png
4,BCC,PAT_684_1302_588.png


In [4]:
data['img_id'].values

array(['PAT_1516_1765_530.png', 'PAT_46_881_939.png',
       'PAT_1545_1867_547.png', ..., 'PAT_1343_1217_404.png',
       'PAT_326_690_823.png', 'PAT_1714_3189_989.png'], dtype=object)

In [5]:

# Function to load images
def load_images(img_paths, img_size=(224, 224)):
    images = []
    for path in img_paths:
        img = tf.keras.preprocessing.image.load_img(path, target_size=img_size)
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = img / 255.0  # Normalize to [0, 1]
        images.append(img)
    return np.array(images)

# Load images
img_paths = "/kaggle/input/pad-ufes-20/PAD-UFES-20/Dataset"+"/"+data['img_id'].values
images = load_images(img_paths)

# Extract labels
labels = data['diagnostic'].values

# One-hot encode the labels
labels = pd.get_dummies(labels).values

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)
#train_df, val_df = train_test_split(data, test_size=0.2, random_state=42)


In [6]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load the VGG16 model pre-trained on ImageNet, excluding the top fully-connected layers
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the VGG16 base model layers
vgg16_base.trainable = False

# Define the input layer
inputs = Input(shape=(224, 224, 3))

# Add the VGG16 base model
x = vgg16_base(inputs, training=False)

# Add new fully connected layers
x = Flatten()(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(6, activation='softmax')(x)  # Ensure this matches the number of unique classes

# Create the model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 6)              │        24,582 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,285,126 (512.26 MB)

 Trainable params: 119,570,438 (456.13 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [7]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2

# Load the ResNet50 model pre-trained on ImageNet data
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
#x = Dropout(0.5)(x)  # Add dropout
#x = Dense(1024, activation='relu', kernel_regularizer=l2(0.01))(x)
predictions = Dense(6, activation='softmax')(x)

# Define the final model
model_res = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model_res.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model_res.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_3[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 26,741,638 (102.01 MB)

 Trainable params: 26,688,518 (101.81 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [ ]:
history = model_res.fit_generator(X_train, y_train, epochs=20, validation_data=(X_val, y_val), batch_size=32)

Epoch 1/20


I0000 00:00:1723130193.222912     104 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1723130193.328400     104 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4044 - loss: 9.5192   

W0000 00:00:1723130251.397928     104 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1723130256.987341     106 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


58/58 ━━━━━━━━━━━━━━━━━━━━ 159s 1s/step - accuracy: 0.4052 - loss: 9.4787 - val_accuracy: 0.3848 - val_loss: 7.8610
Epoch 2/20


W0000 00:00:1723130263.673678     106 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


58/58 ━━━━━━━━━━━━━━━━━━━━ 18s 318ms/step - accuracy: 0.5592 - loss: 3.1741 - val_accuracy: 0.0935 - val_loss: 5.0076
Epoch 3/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 20s 312ms/step - accuracy: 0.6005 - loss: 1.8554 - val_accuracy: 0.1087 - val_loss: 4.3735
Epoch 4/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 18s 304ms/step - accuracy: 0.6486 - loss: 1.4118 - val_accuracy: 0.1087 - val_loss: 6.5429
Epoch 5/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 18s 304ms/step - accuracy: 0.6803 - loss: 1.1479 - val_accuracy: 0.1087 - val_loss: 2.8474
Epoch 6/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 18s 310ms/step - accuracy: 0.7171 - loss: 0.9116 - val_accuracy: 0.1087 - val_loss: 6.8460
Epoch 7/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 18s 310ms/step - accuracy: 0.7463 - loss: 0.8563 - val_accuracy: 0.1087 - val_loss: 5.8119
Epoch 8/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 18s 309ms/step - accuracy: 0.7601 - loss: 0.7723 - val_accuracy: 0.1087 - val_loss: 3.1919
Epoch 9/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 18s 306ms/step - accuracy: 0.7975 - loss: 0.6463 - val_accuracy: 0.108